# Experiments with GP pushing and then simulated annealing for finalization

In [51]:
%load_ext autoreload
%autoreload 2
from DLC_for_WBFM.utils.projects.finished_project_data import finished_project_data
import numpy as np
import matplotlib.pyplot as plt
import pickle
import napari, zarr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"
dat = finished_project_data.load_final_project_data_from_config(fname)

In [ ]:
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\2-training_data\raw\match_dat.pickle"
with open(fname, 'rb') as f:
    matches = pickle.load(f)
    
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\2-training_data\raw\frame_dat.pickle"
with open(fname, 'rb') as f:
    frames = pickle.load(f)

# Visualize the pushed neurons on the next frame

In [ ]:
pair = (0, 1)
print(matches[pair])
print(frames[pair[0]], frames[pair[1]])

In [ ]:
# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

In [ ]:
# Get gp pushed locations

gp_zxy = matches[pair].gp_pushed_locations
# Replace z coordinate, which is somehow crazy
gp_zxy[:, 0] = frames[pair[0]].neuron_locs[:, 0]
gp_zxy = gp_zxy[:, [0,2,1]]

affine_zxy = matches[pair].affine_pushed_locations
# Replace z coordinate, which is somehow crazy
# affine_zxy[:, 0] = frames[pair[0]].neuron_locs[:, 0]
affine_zxy = affine_zxy[:, [0,2,1]]

In [ ]:
# Split by "has match"
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}

# F1 matches
has_match_ind = set(f0_to_f1_matches.values())
target_xyz_matched = [row for i, row in enumerate(target_zxy) if i in has_match_ind]
target_xyz_unmatched = [row for i, row in enumerate(target_zxy) if i not in has_match_ind]

# F0 matches
has_match_ind = set(f0_to_f1_matches.keys())

gp_xyz_matched = [row for i, row in enumerate(gp_zxy) if i in has_match_ind]
gp_xyz_unmatched = [row for i, row in enumerate(gp_zxy) if i not in has_match_ind]

affine_xyz_matched = [row for i, row in enumerate(affine_zxy) if i in has_match_ind]
affine_xyz_unmatched = [row for i, row in enumerate(affine_zxy) if i not in has_match_ind]

In [ ]:
# Plot using napari
v = napari.view_image(dat.red_data[pair[1]], ndisplay=3)
# v.add_points(gp_xyz_matched, size=3, face_color='green', n_dimensional=True)
v.add_points(gp_xyz_unmatched, size=3, face_color='red', n_dimensional=True)
# v.add_points(affine_xyz_matched, size=3, face_color='green', symbol='x', n_dimensional=True)
v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True)
v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True)

In [ ]:

# Get gradient images (not in z)
# vol = dat.red_data[pair[1], ...]
# vol_filtered = gaussian(vol, sigma=1)
# thresh = 1.5*np.mean(vol_filtered)
# mask = vol_filtered > thresh
# vol_gradient_x = sobel_h(vol_filtered, mask)
# vol_gradient_y = sobel_v(vol_filtered, mask)

# Actually do it (no annealing)

In [ ]:
# from skimage.filters import gaussian, sobel_h, sobel_v
from scipy.spatial import KDTree

In [ ]:
pair = (1, 2)

In [ ]:
# Get initial affine locations
affine_zxy = matches[pair].affine_pushed_locations
affine_zxy = affine_zxy[:, [0,2,1]]

# Get image
vol = dat.red_data[pair[1], ...]
# vol_filtered = gaussian(vol, sigma=1)

# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

In [ ]:
# Only check the ones with no match
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}
has_match_ind = list(f0_to_f1_matches.keys())
no_match_ind = list(range(frames[pair[0]].num_neurons()))
[no_match_ind.remove(i) for i in has_match_ind]

# Set up kdtree to check nearest neighbor
tree = KDTree(target_zxy)

# Parameters
brightness_thresh = 25
distance_thresh = 5.0

In [ ]:
new_f1_zxy = []
new_matches = []
next_f1_ind = frames[pair[1]].num_neurons() + 1

for ind_f0 in no_match_ind:
    pushed_zxy = affine_zxy[ind_f0, :]
    z, x, y = int(pushed_zxy[0]), int(pushed_zxy[1]), int(pushed_zxy[2])
    brightness = vol[z, x, y]
    
    nn_dist, _ = tree.query(pushed_zxy, k=1)
    
    to_keep = (brightness > brightness_thresh) and (nn_dist > distance_thresh)
    print(to_keep, brightness, nn_dist)
    if to_keep:
        new_matches.append([ind_f0, next_f1_ind, 1.0])
        next_f1_ind += 1
        new_f1_zxy.append(pushed_zxy)

print(f"Added {len(new_matches)} neurons")

In [ ]:
# Plot using napari
v = napari.view_image(dat.red_data[pair[1]], ndisplay=3)
v.add_points(new_f1_zxy, size=3, face_color='green', symbol='x', n_dimensional=True)
v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True)
v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True)

In [ ]:
f0, f1 = frames[pair[0]], frames[pair[1]]

In [ ]:
len(f0.keypoint_locs)

# Use piecewise affine with only matched neurons

In [ ]:
from skimage.transform import PiecewiseAffineTransform, warp

In [ ]:
pair = (0, 1)
print(matches[pair])
print(frames[pair[0]], frames[pair[1]])

In [ ]:
m = matches[pair]
f0, f1 = frames[pair[0]], frames[pair[1]]

In [ ]:
f0_to_f1 = m.get_f0_to_f1_dict()

In [ ]:
src_matched = np.array([f0.keypoint_locs[k,[2,1]] for k in f0_to_f1.keys()])
src_all_xy = np.array(f0.keypoint_locs[:,[2,1]])
src_z = np.array(f0.keypoint_locs[:,[0]])

dst_matched = np.array([f1.keypoint_locs[v,[2,1]] for v in f0_to_f1.values()])
dst_all_xy = np.array(f0.keypoint_locs[:,[2,1]])
dst_z = np.array(f0.keypoint_locs[:,[0]])

In [ ]:

tform = PiecewiseAffineTransform()
tform.estimate(src_matched, dst_matched)

In [ ]:
transformed = tform.inverse(dst_all_xy)
# src_transformed = tform.inverse(src)

transformed = np.where(transformed==-1, np.nan, transformed)

transformed = np.hstack([dst_z, transformed]) #[:, [0,2,1]]

In [ ]:
# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

affine_zxy = matches[pair].affine_pushed_locations
affine_zxy = affine_zxy[:, [0,2,1]]

# Split by "has match"
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}

# F1 matches
has_match_ind = set(f0_to_f1_matches.values())
target_xyz_matched = [row for i, row in enumerate(target_zxy) if i in has_match_ind]
target_xyz_unmatched = [row for i, row in enumerate(target_zxy) if i not in has_match_ind]

# F0 matches
has_match_ind = set(f0_to_f1_matches.keys())

affine_xyz_matched = [row for i, row in enumerate(affine_zxy) if i in has_match_ind]
affine_xyz_unmatched = [row for i, row in enumerate(affine_zxy) if i not in has_match_ind]

In [ ]:
# # Plot using napari
# v = napari.view_image(dat.red_data[pair[0]], ndisplay=3, name='f0_red')
# v.add_image(dat.red_data[pair[1]], name='f1_red')
# v.add_points(transformed, size=3, face_color='red', n_dimensional=True, name='piecewise_affine_transformed')
# v.add_points(affine_xyz_matched, size=3, face_color='green', symbol='x', n_dimensional=True, name='f0_affine_matched')
# # v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
# v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True, name='f1_matched')
# v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True, name='f1_unmatched')

In [ ]:

# tform.estimate(dst, src)
# src_transformed = tform.inverse(src)
# # src_transformed = tform.inverse(f0.keypoint_locs[:,[2,1]])

# src_transformed[:3], dst[:3], src[:3]

# Piecewise affine, but add additional ORB features

In [ ]:
from skimage.transform import PiecewiseAffineTransform, warp

In [ ]:
pair = (0, 1)
pair_obj = matches[pair]
print(pair_obj)
f0, f1 = frames[pair[0]], frames[pair[1]]
print(f0, f1)

In [ ]:
# Update Frame classes, if old style
pair_obj.frame0, pair_obj.frame1 = f0, f1

In [ ]:
video_dat = dat.red_data[pair[0],...]

f0.detect_non_neuron_keypoints(video_dat, append_to_existing_keypoints=True)
f0.encode_all_neurons(video_dat, z_depth=3)
f0.build_nontrivial_keypoint_to_neuron_mapping(neuron_feature_radius=3)

In [ ]:
%debug

In [ ]:
m = matches[pair]
f0, f1 = frames[pair[0]], frames[pair[1]]

In [ ]:
f0_to_f1 = m.get_f0_to_f1_dict()

In [ ]:
src_matched = np.array([f0.neuron_locs[k,[2,1]] for k in f0_to_f1.keys()])
src_all_xy = np.array(f0.neuron_locs[:,[2,1]])
src_z = np.array(f0.neuron_locs[:,[0]])

dst_matched = np.array([f1.neuron_locs[v,[2,1]] for v in f0_to_f1.values()])
dst_all_xy = np.array(f0.neuron_locs[:,[2,1]])
dst_z = np.array(f0.neuron_locs[:,[0]])

In [ ]:
tform = PiecewiseAffineTransform()
tform.estimate(src_matched, dst_matched)

In [ ]:
transformed = tform.inverse(dst_all_xy)
# src_transformed = tform.inverse(src)

transformed = np.where(transformed==-1, np.nan, transformed)

transformed = np.hstack([dst_z, transformed]) #[:, [0,2,1]]

In [ ]:
# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

affine_zxy = matches[pair].affine_pushed_locations
affine_zxy = affine_zxy[:, [0,2,1]]

# Split by "has match"
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}

# F1 matches
has_match_ind = set(f0_to_f1_matches.values())
target_xyz_matched = [row for i, row in enumerate(target_zxy) if i in has_match_ind]
target_xyz_unmatched = [row for i, row in enumerate(target_zxy) if i not in has_match_ind]

# F0 matches
has_match_ind = set(f0_to_f1_matches.keys())

affine_xyz_matched = [row for i, row in enumerate(affine_zxy) if i in has_match_ind]
affine_xyz_unmatched = [row for i, row in enumerate(affine_zxy) if i not in has_match_ind]

In [ ]:
# Plot using napari
v = napari.view_image(dat.red_data[pair[0]], ndisplay=3, name='f0_red')
v.add_image(dat.red_data[pair[1]], name='f1_red')
v.add_points(transformed, size=3, face_color='red', n_dimensional=True, name='piecewise_affine_transformed')
v.add_points(affine_xyz_matched, size=3, face_color='green', symbol='x', n_dimensional=True, name='f0_affine_matched')
# v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True, name='f1_matched')
v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True, name='f1_unmatched')

# Scratch: sanity checks for the frames

In [ ]:
fname = r"C:\dlc_stacks\Test_project\2-training_data\all_tracklets.h5"
import pandas as pd

df = pd.read_hdf(fname)

In [ ]:
df

In [ ]:
df.diff()[1:]


In [ ]:
all_neurons = list(df.columns.levels[0])
for n in all_neurons:
    zd = df.diff()[1:][n]['z']
    if all(abs(zd) > 2):
        print(n, list(zd))

In [ ]:
np.quantile(np.arange(10), 0.9)